### Body Fat Prediction

In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Importing Data

In [129]:
bodyfat = pd.read_csv("C://Users//Ambarish Deb//bodyfatwebapp//data and notebook//bodyfat.csv")
bodyfat.head()

In [130]:
bodyfat.dtypes

### Data Cleaning

In [131]:
bodyfat.isnull().sum()

In [132]:
import random
from scipy.stats import norm, skew

In [133]:
def randomcolor():
    r = random.random()
    b = random.random()
    g = random.random()
    rgb = [r,g,b]
    return rgb

In [134]:
X = bodyfat.drop('BodyFat',axis = 1)
for x in X.columns:
    sns.boxplot(bodyfat[x],color = randomcolor())
    plt.title("Spread of "+x + " variable")
    plt.show()

The columns have a few outlying points at most. So we don't need to treat them for outliers as such.

In [135]:
for x in X.columns:
    skewness = skew(bodyfat[x])
    sns.distplot(bodyfat[x], fit = norm, color = randomcolor())
    plt.title("Skewness of variable "+ x+" : "+str(skewness))
    plt.show()

Compared to other columns, height and ankle are more skewed compared to others. Fixing them by transforming them

In [136]:
#from scipy.stats import boxcox

In [137]:
bodyfat["Height_normal"] = np.square(bodyfat["Height"])
bodyfat["Ankle_normal"] = np.log(bodyfat["Ankle"])

In [138]:
for x in ["Height_normal","Ankle_normal"]:
    skewness = skew(bodyfat[x])
    sns.distplot(bodyfat[x], fit = norm, color = randomcolor())
    plt.title("Skewness of variable "+ x+" : "+str(skewness))
    plt.show()

In [139]:
bodyfat["Height_normal"] = np.square(bodyfat["Height_normal"])
bodyfat["Ankle_normal"] = np.log(bodyfat["Ankle_normal"])
for x in ["Height_normal","Ankle_normal"]:
    skewness = skew(bodyfat[x])
    sns.distplot(bodyfat[x], fit = norm, color = randomcolor())
    plt.title("Skewness of variable "+ x+" : "+str(skewness))
    plt.show()

### EDA

Checking Correlation between bodyfat and X variables

In [140]:
from scipy.stats import pearsonr

In [141]:
for x in X.columns:
    corr, _ = pearsonr(bodyfat[x], bodyfat.BodyFat)
    sns.scatterplot(x=bodyfat[x], y=bodyfat.BodyFat, color = randomcolor())
    plt.title('Pearsons correlation: %.3f' % corr)
    plt.show()

Except height and density, all other variables are positively correlated with bodyfat.
For those variables having a pearson coefficient of >=0.5 we can say that increase in them correlates to increase in bodyfat and vice versa.
Similarly those variables having a pearson coefficient of <=-0.5 we can say that increase in them correlates to decrease in bodyfat and vice versa.

In [142]:
sns.pairplot(bodyfat.drop("BodyFat",axis = 1),diag_kind='kde')
plt.show()

In [143]:
f,ax = plt.subplots(figsize=(20, 20))
sns.heatmap(bodyfat.drop("BodyFat",axis = 1).corr(), annot = True, fmt= '.2f')
plt.show()

In [144]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
def compute_vif(df,considered_features):
    
    X = df[considered_features]
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

In [145]:
compute_vif(bodyfat,(X.columns))

Variables with high VIF are Weight, Abdomen, Hip, Thigh.
We'll remove them one by one until the vif of all other variables settles to below 5.

In [146]:
considered_features = list(X.columns)
considered_features.remove('Weight')
compute_vif(bodyfat,considered_features)

In [147]:
considered_features.remove('Abdomen')
compute_vif(bodyfat,considered_features)

In [148]:
considered_features.remove('Hip')
compute_vif(bodyfat,considered_features)

In [149]:
considered_features.remove('Thigh')
compute_vif(bodyfat,considered_features)

### Data Preparation, train test split

In [150]:
from sklearn.model_selection import train_test_split

In [151]:
X = bodyfat.drop(["BodyFat","Height_normal","Ankle_normal"],axis = 1)
X_vif_adj = bodyfat[considered_features]
Y = bodyfat.BodyFat

In [152]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_vif_adj_train, X_vif_adj_test, Y_vif_adj_train, Y_vif_adj_test = train_test_split(X_vif_adj, Y, test_size=0.3, random_state=42)

### Regression Modeling

In [153]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [154]:
def linear_model(model, X_train,Y_train, X_test, Y_test):
    model.fit(X_train,Y_train)
    Y_pred = model.predict(X_test)
    score = model.score(X_test, Y_test)
    rmse = mean_squared_error(Y_test, Y_pred, squared=False)
    r2 = r2_score(Y_test, Y_pred)
    adj_r2 = 1 - (1-r2_score(Y_test, Y_pred))*(len(Y)-1)/(len(Y)-X.shape[1]-1)
    print(f"RMSE = {rmse}")
    print(f"R^2 Score = {r2}")
    print(f"Adjusted R^2 Score = {adj_r2}")
    return model

### K Neighbours Regressor

In [155]:
knr1 = linear_model(KNeighborsRegressor(),X_train,Y_train, X_test, Y_test)

In [156]:
knr1_vif_adj = linear_model(KNeighborsRegressor(),X_vif_adj_train,Y_train, X_vif_adj_test, Y_test)

### Linear Regression

In [157]:
lr1 = linear_model(LinearRegression(),X_train,Y_train, X_test, Y_test)

In [158]:
lr1_vif_adj = linear_model(LinearRegression(),X_vif_adj_train,Y_train, X_vif_adj_test, Y_test)

### Decision Tree Regressor

In [159]:
dtr1 = linear_model(DecisionTreeRegressor(),X_train,Y_train, X_test, Y_test)

In [160]:
dtr1_vif_adj = linear_model(DecisionTreeRegressor(),X_vif_adj_train,Y_train, X_vif_adj_test, Y_test)

### Random Forest Regressor

In [161]:
rfr1 = linear_model(RandomForestRegressor(),X_train,Y_train, X_test, Y_test)

In [162]:
rfr1_vif_adj = linear_model(RandomForestRegressor(),X_vif_adj_train,Y_train, X_vif_adj_test, Y_test)

#### Hyperparameter Tuning

In [163]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV,RandomizedSearchCV

In [164]:
rfr = RandomForestRegressor()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [165]:
rscv = RandomizedSearchCV(estimator = rfr1_vif_adj, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rscv.fit(X_vif_adj_train,Y_train)
rscv.best_params_

In [166]:
hrscv = HalvingRandomSearchCV(estimator = rfr1_vif_adj, param_distributions = random_grid, cv = 3, verbose=2, random_state=42, n_jobs = -1)
hrscv.fit(X_vif_adj_train,Y_train)
hrscv.best_params_

In [167]:
ht_rfr1_vif_adj = linear_model(RandomForestRegressor(n_estimators= 1800,min_samples_split=2,min_samples_leaf= 2,max_features='auto',max_depth= 80,bootstrap= True),X_vif_adj_train,Y_train, X_vif_adj_test, Y_test)

In [168]:
ht_rfr1_vif_adj_1 = linear_model(RandomForestRegressor(n_estimators= 2000,min_samples_split=2,min_samples_leaf= 2,max_features='auto',max_depth= 50,bootstrap= False),X_vif_adj_train,Y_train, X_vif_adj_test, Y_test)

Out of all the models trained, Linear Regression with VIF adjusted features gave the best R^2 score. Hence we'll proceed with exporting that model.

In [169]:
### Saving the model for deployment using joblib
import joblib 
joblib.dump(lr1_vif_adj,'C://Users//Ambarish Deb//bodyfatwebapp//bodyfat_linearreg.sav')

In [170]:
#considered_features

In [171]:
#lr1_vif_adj.predict([[1.0316,65,65.75,40.8,106.4,38.1,24.0,35.9,30.5,19.1]])

In [172]:
#X_vif_adj_train

In [173]:
#Y_train